In [107]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/data.csv')



In [109]:
# For full dataset

df['key_skills'] = df['key_skills'].fillna('')
df['key_skills'] = df['key_skills'].str.replace('{','').str.replace('}','').str.replace('"','')
df['key_skills_list'] = df['key_skills'].str.split(',')
df['key_skills'] = df['key_skills'].str.replace(';',',')
df['key_skills'] = df['key_skills_list'].str.join(';')


## description cleaning

import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

df['description'] = df['description'].apply(cleanhtml)

df_train, df_test = df.iloc[:700000], df.iloc[700000:]

/tmp/ipykernel_401890/3459542558.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['key_skills'] = df['key_skills'].str.replace('{','').str.replace('}','').str.replace('"','')


In [110]:
# For IT dataset
# df = df[df['description'].apply(len)<2000]
# df_train, df_test = df.iloc[:45000], df.iloc[45000:]

In [111]:
df_train['key_skills_list'] = df_train['key_skills'].str.split(';')

/tmp/ipykernel_401890/1578208901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['key_skills_list'] = df_train['key_skills'].str.split(';')


In [112]:
all_key_skills = []
for key_skills in df_train['key_skills_list']:
    all_key_skills+=key_skills
all_key_skills = pd.Series(all_key_skills)

In [113]:
all_key_skills_counts = all_key_skills.value_counts()

In [117]:
all_key_skills_counts_filtred = all_key_skills_counts[all_key_skills_counts>100]

In [118]:
from flashtext.keyword import KeywordProcessor
keyword_processor = KeywordProcessor()
for key_skill in all_key_skills_counts_filtred.index:
    keyword_processor.add_keyword(key_skill)
    keyword_processor.add_keyword(key_skill)
    
    
keywords_found = keyword_processor.extract_keywords('Обязанности:  анализ документации и работа со статистикой; разработка нагрузочных скриптов (Java или C#) ; запуск тестов и анализ результатов, написание отчетов и составление документации; А также: разработка заглушек и эмуляторов, разработка утилит для генерации данных, работа с базами данных.  Требования:  уверенное знание одного из языков программирование: C/C++, Java, C#, Phython; умение работать с базами данных на уровне написания простейших SQL-запросов; знание архитектур приложений (толстый/тонкий клиент); возможность работать полный рабочий день.  Условия:  работа в молодом активном коллективе; конкурентная заработная плата, оформление по ТК РФ; постоянное обучение внутри компании; возможность профессионального роста; участие в тестировании ПО в крупных российских банках.    Первый этап - собеседование по онлайн.')
print(keywords_found)

['Анализ', 'работа со статистикой', 'ПТО', 'Java', 'C#', 'Анализ', 'Работа с базами данных', 'программирование', 'C/C++', 'Java', 'C#', 'Работа', 'SQL', 'ТУ', 'Работа', 'рабочий', 'Работа', 'ТК РФ', 'Обучение', 'Банк', 'Собеседование']


In [119]:
%%time
df_test['extracted_key_skills'] = df_test['description'].apply(lambda x: ';'.join(list(set(keyword_processor.extract_keywords(x)))))

CPU times: user 1min 46s, sys: 174 ms, total: 1min 47s
Wall time: 1min 47s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [120]:
df_test.to_csv('../data/full_dataset_test_search.csv', index = False)